In [ ]:
!pip install -q pyomo
 
#we add cbc
!apt-get install -y -qq coinor-cbc
 
#we add ipopt
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
 
#we add glpk
!sudo apt install libglpk-dev python3.8-dev libgmp3-dev
!apt-get install -y -qq glpk-utils
 
#we add couenne
!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
!unzip -o -q couenne-linux64
 
#we add bonmin
!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
!unzip -o -q bonmin-linux64
 
#let us also try commercial solvers
!pip install cplex
!pip install xpress
!pip install Mosek

     |████████████████████████████████| 9.4MB 4.0MB/s 
     |████████████████████████████████| 51kB 3.8MB/s 
     |████████████████████████████████| 256kB 42.6MB/s 
     |████████████████████████████████| 163kB 44.0MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coi

In [ ]:
%matplotlib inline
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import logging
from IPython.display import Markdown

In [ ]:
cbc_solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc')
glpk_solver = pyo.SolverFactory('glpk')
ipopt_solver = pyo.SolverFactory('ipopt', executable='/content/ipopt')

# Exercise 1: Farmer's problem and some of its variants

In the farmer's problem (please refer to the lecture notes or slides for a full description of the problem), a farmer has to allocate $500$ acres of land to three different types of crop aiming to maximize her profit. 

Recall that:

*   Planting one acre of wheat, corn and beet costs 150, 230 and 260 euro, respectively.

*   At least 200 tons of wheat and 240 tons of corn are needed for cattle feed, which can be purchased from a wholesaler if not harvested by her farm.

*   Up to 6000 tons of sugar beets can be sold for 36 euro per ton, while any additional amounts can be sold for 10 euro per ton.

*   Any wheat or corn not used for the cattle can be sold at 170 euro and 150 euro per ton of wheat and corn, respectively. The wholesaler sells the wheat or corn at a higher price, namely 238 euro and 210 euro per ton, respectively.

In her decision, the farmer considers three weather scenarios, each one having a different yield in tons/acre per crop type as summarized by the following table.

| Scenario | Yield for weath | Yield for corn | Yield for beets |
| :-: | :-: | :-: | :-: |
| Good weather | 3 | 3.6 | 24 |
| Average weather | 2.5 | 3 | 20 |
| Bad weather | 2 | 2.4 | 16 |

We first consider the case in which all the prices are fixed and not weather-dependent. The following table summarizes them (unit=euro/ton)

| Selling price for weath | Selling price for corn | Purchasing price for weath | Purchasing price for corn | Selling price for beets ($\leq 6000$) | Selling price for beets ($> 6000$) |
| :-: | :-: | :-: | :-: | :-: | :-: |
| 170 | 150 | 238 | 210 | 36 | 10 |

(a) Implement the extensive form of stochastic LP corresponding to the farmer's problem in Pyomo and solve it.


In [ ]:
model = pyo.ConcreteModel()

model.crops = pyo.Set(initialize=['W', 'C', 'S'])
model.totalacres = 500
model.factor_H = 1.2 # to obtain the yields in the good weather (high yield) case by multiplying the average ones
model.factor_L = 0.8 # to obtain the yields in the bad weather (low yield) case by multiplying the average ones

# first stage variables
model.plant = pyo.Var(model.crops, within=pyo.NonNegativeReals) 

# first stage constraint
model.total_acres = pyo.Constraint(expr=pyo.summation(model.plant) <= model.totalacres)

model.scenarios = pyo.Set(initialize=['H', 'M', 'L'])  # high, medium, and low yield scenarios

# second stage variables (labelled as H,M,L depending on the scenario)
# the sell_extra variables refer to the amount of beets to be sold beyond the 6000 threshold, if any

model.sell_H = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_H = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_H = pyo.Var(within=pyo.NonNegativeReals) 

model.sell_M = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_M = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_M = pyo.Var(within=pyo.NonNegativeReals)

model.sell_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_L = pyo.Var(within=pyo.NonNegativeReals)

# second stage constraints
model.feed_cattle_W_H = pyo.Constraint(expr=model.plant['W'] * 2.5 * model.factor_H - model.sell_H['W'] + model.buy_H['W'] >= 200)
model.feed_cattle_C_H = pyo.Constraint(expr=model.plant['C'] * 3 * model.factor_H - model.sell_H['C'] + model.buy_H['C'] >= 240)
model.sell_S_extra_H = pyo.Constraint(expr=model.plant['S'] * 20 * model.factor_H >= model.sell_H['S'] + model.sell_extra_H)
model.sell_S_H = pyo.Constraint(expr=model.sell_H['S'] <= 6000)
model.nobuy_H = pyo.Constraint(expr=model.buy_H['S'] == 0)

model.feed_cattle_W_M = pyo.Constraint(expr=model.plant['W'] * 2.5 - model.sell_M['W'] + model.buy_M['W'] >= 200)
model.feed_cattle_C_M = pyo.Constraint(expr=model.plant['C'] * 3 - model.sell_M['C'] + model.buy_M['C'] >= 240)
model.sell_S_extra_M = pyo.Constraint(expr=model.plant['S'] * 20 >= model.sell_M['S'] + model.sell_extra_M)
model.sell_S_M = pyo.Constraint(expr=model.sell_M['S'] <= 6000)
model.nobuy_M = pyo.Constraint(expr=model.buy_M['S'] == 0)

model.feed_cattle_W_L = pyo.Constraint(expr=model.plant['W'] * 2.5 * model.factor_L - model.sell_L['W'] + model.buy_L['W'] >= 200)
model.feed_cattle_C_L = pyo.Constraint(expr=model.plant['C'] * 3 * model.factor_L - model.sell_L['C'] + model.buy_L['C'] >= 240)
model.sell_S_extra_L = pyo.Constraint(expr=model.plant['S'] * 20 * model.factor_L >= model.sell_L['S'] + model.sell_extra_L)
model.sell_S_L = pyo.Constraint(expr=model.sell_L['S'] <= 6000)
model.nobuy_L = pyo.Constraint(expr=model.buy_L['S'] == 0)

def first_stage_profit(model):
    return -model.plant["W"] * 150 - model.plant["C"] * 230 - model.plant["S"] * 260

model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

def second_stage_profit(model):
    total_H = -model.buy_H['W'] * 238 - model.buy_H['C'] * 210 + 36 * model.sell_H['S'] + 10 * model.sell_extra_H + model.sell_H['W'] * 170 + model.sell_H['C'] * 150
    total_M = -model.buy_M['W'] * 238 - model.buy_M['C'] * 210 + 36 * model.sell_M['S'] + 10 * model.sell_extra_M + model.sell_M['W'] * 170 + model.sell_M['C'] * 150
    total_L = -model.buy_L['W'] * 238 - model.buy_L['C'] * 210 + 36 * model.sell_L['S'] + 10 * model.sell_extra_L + model.sell_L['W'] * 170 + model.sell_L['C'] * 150
    return (total_H + total_M + total_L)/3.0

model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

def total_profit(model):
    return model.first_stage_profit + model.second_stage_profit

model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

result = cbc_solver.solve(model)
display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
display(Markdown(f"**Solution:**"))
display(Markdown(f"(land allocation) $x_1 = {model.plant['W'].value:.1f}$, $x_2 = {model.plant['C'].value:.1f}$, $x_3 = {model.plant['S'].value:.1f}$"))
display(Markdown(f"(recourse sell action high yield) $w_1 = {model.sell_H['W'].value:.1f}$, $w_2 = {model.sell_H['C'].value:.1f}$, $w_3 = {model.sell_H['S'].value:.1f}$, $w_4 = {model.sell_extra_H.value:.1f}$"))
display(Markdown(f"(recourse purchase action high yield) $y_1 = {model.buy_H['W'].value:.1f}$, $y_2 = {model.buy_H['C'].value:.1f}$, $y_3 = {model.buy_H['S'].value:.1f}$"))
display(Markdown(f"(recourse sell action medium yield) $w_1 = {model.sell_M['W'].value:.1f}$, $w_2 = {model.sell_M['C'].value:.1f}$, $w_3 = {model.sell_M['S'].value:.1f}$, $w_4 = {model.sell_extra_M.value:.1f}$"))
display(Markdown(f"(recourse purchase action medium yield) $y_1 = {model.buy_M['W'].value:.1f}$, $y_2 = {model.buy_M['C'].value:.1f}$, $y_3 = {model.buy_M['S'].value:.1f}$"))
display(Markdown(f"(recourse sell action low yield) $w_1 = {model.sell_L['W'].value:.1f}$, $w_2 = {model.sell_L['C'].value:.1f}$, $w_3 = {model.sell_L['S'].value:.1f}$, $w_4 = {model.sell_extra_L.value:.1f}$"))
display(Markdown(f"(recourse purchase action low yield) $y_1 = {model.buy_L['W'].value:.1f}$, $y_2 = {model.buy_L['C'].value:.1f}$, $y_3 = {model.buy_L['S'].value:.1f}$"))
display(Markdown(f"**Maximizes objective value to:** ${model.total_expected_profit():.0f}$€"))

**Solver status:** *ok, optimal*

**Solution:**

(land allocation) $x_1 = 170.0$, $x_2 = 80.0$, $x_3 = 250.0$

(recourse sell action high yield) $w_1 = 310.0$, $w_2 = 48.0$, $w_3 = 6000.0$, $w_4 = 0.0$

(recourse purchase action high yield) $y_1 = 0.0$, $y_2 = 0.0$, $y_3 = 0.0$

(recourse sell action medium yield) $w_1 = 225.0$, $w_2 = 0.0$, $w_3 = 5000.0$, $w_4 = 0.0$

(recourse purchase action medium yield) $y_1 = 0.0$, $y_2 = 0.0$, $y_3 = 0.0$

(recourse sell action low yield) $w_1 = 140.0$, $w_2 = 0.0$, $w_3 = 4000.0$, $w_4 = 0.0$

(recourse purchase action low yield) $y_1 = 0.0$, $y_2 = 48.0$, $y_3 = 0.0$

**Maximizes objective value to:** $108390$€

Please note a second way to create this model which makes use of `pyomo` `blocks`. For an explanation of `blocks` refer to chapter 8 of the `pyomo` book that you may [download](https://vu.on.worldcat.org/oclc/988749903) from the VU library.

We leave as an exercise to redo the rest of the questions using `blocks` and see how that may help.

Note that for b) you will need to provide prices as parameters to the blocks. 
Note as well that is the resolution below already had done that, instead of typing the prices as numerical constants, then the changes would have been immediate. 

In [ ]:
m = pyo.ConcreteModel()

m.crops = pyo.Set(initialize=['W', 'C', 'S'])
m.totalacres = 500

# first stage variables
m.plant = pyo.Var(m.crops, within=pyo.NonNegativeReals) 

# first stage constraint
m.total_acres = pyo.Constraint(expr=pyo.summation(m.plant) <= m.totalacres)

m.scenarios = pyo.Set(initialize=['H', 'M', 'L'])  # high, medium, and low yield scenarios

# this could be a dataframe, or any other data source
nominal_yields = { 'W' : 2.5, 'C' :   3, 'S' : 20 }
factor_yields  = { 'M' : 1, 'H' : 1.2, 'L' : 0.8 }
m.yields = { s : { c : nominal_yields[c]*factor_yields[s] for c in m.crops } for s in m.scenarios }

def scenario_block(b,s):
  b.yields     = pyo.Param(m.crops,initialize=m.yields[s])
  b.sell       = pyo.Var(m.crops, within=pyo.NonNegativeReals)
  b.buy        = pyo.Var(m.crops, within=pyo.NonNegativeReals)
  b.sell_extra = pyo.Var(within=pyo.NonNegativeReals) 
  b.sell_S     = pyo.Constraint(expr=b.sell['S'] <= 6000)
  b.nobuy      = pyo.Constraint(expr=b.buy['S'] == 0)
  b.profit     = pyo.Expression(expr= -238*b.buy['W'] -210*b.buy['C'] +36*b.sell['S'] + 10*b.sell_extra + 170*b.sell['W'] + 150*b.sell['C'])

m.scenario = pyo.Block( m.scenarios, rule=scenario_block)

# second stage (linking) constraints
m.feed_cattle_W = pyo.Constraint(m.scenarios, rule = lambda m, s : m.plant['W'] * m.scenario[s].yields['W'] - m.scenario[s].sell['W'] + m.scenario[s].buy['W'] >= 200)
m.feed_cattle_C = pyo.Constraint(m.scenarios, rule = lambda m, s : m.plant['C'] * m.scenario[s].yields['C'] - m.scenario[s].sell['C'] + m.scenario[s].buy['C'] >= 240)
m.sell_S_extra  = pyo.Constraint(m.scenarios, rule = lambda m, s : m.plant['S'] * m.scenario[s].yields['S'] >= m.scenario[s].sell['S'] + m.scenario[s].sell_extra )

m.first_stage_profit = pyo.Expression( expr = -150*m.plant["W"] -230*m.plant["C"] -260*m.plant["S"] )
m.total_expected_profit = pyo.Objective( rule = lambda m : m.first_stage_profit + sum(m.scenario[s].profit for s in m.scenarios)/3, sense=pyo.maximize )

result = cbc_solver.solve(m)
display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
display(Markdown(f"**Solution:**"))
display(Markdown(f"(land allocation) $x_1 = {m.plant['W'].value:.1f}$, $x_2 = {m.plant['C'].value:.1f}$, $x_3 = {m.plant['S'].value:.1f}$"))
spellout = { 'H' : 'high', 'M' : 'medium', 'L' : 'low' }
for s in m.scenarios:
  display(Markdown(f"(recourse sell action {spellout[s]} yield) $w_1 = {m.scenario[s].sell['W'].value:.1f}$, $w_2 = {m.scenario[s].sell['C'].value:.1f}$, $w_3 = {m.scenario[s].sell['S'].value:.1f}$, $w_4 = {m.scenario[s].sell_extra.value:.1f}$"))
  display(Markdown(f"(recourse purchase action {spellout[s]} yield) $y_1 = {m.scenario[s].buy['W'].value:.1f}$, $y_2 = {m.scenario[s].buy['C'].value:.1f}$, $y_3 = {m.scenario[s].buy['S'].value:.1f}$"))
display(Markdown(f"**Maximizes objective value to:** ${m.total_expected_profit.expr():.0f}$€"))

**Solver status:** *ok, optimal*

**Solution:**

(land allocation) $x_1 = 170.0$, $x_2 = 80.0$, $x_3 = 250.0$

(recourse sell action high yield) $w_1 = 310.0$, $w_2 = 48.0$, $w_3 = 6000.0$, $w_4 = 0.0$

(recourse purchase action high yield) $y_1 = 0.0$, $y_2 = 0.0$, $y_3 = 0.0$

(recourse sell action medium yield) $w_1 = 225.0$, $w_2 = 0.0$, $w_3 = 5000.0$, $w_4 = 0.0$

(recourse purchase action medium yield) $y_1 = 0.0$, $y_2 = 0.0$, $y_3 = 0.0$

(recourse sell action low yield) $w_1 = 140.0$, $w_2 = 0.0$, $w_3 = 4000.0$, $w_4 = 0.0$

(recourse purchase action low yield) $y_1 = 0.0$, $y_2 = 48.0$, $y_3 = 0.0$

**Maximizes objective value to:** $108390$€

If the weather is good and yields are high for the farmer, they are probably so also for many other farmers. The total supply is thus increasing, which will lower the prices. Assume the prices going down by 10% for corn and wheat when the weather is good and going up by 10% when the weather is bad. These changes in prices affect both sales and purchases of corn and wheat, but sugar beet prices are not affected by yields. The following table summarizes the scenario-dependent prices:

| Scenario | Selling price for weath | Selling price for corn | Purchasing price for weath | Purchasing price for corn | 
| :-: | :-: | :-: | :-: | :-: |
| Good weather | 153| 135| 214| 189|
| Average weather | 170 | 150 | 238 | 210 |
| Bad weather |187| 165| 262| 231|

(b) Implement the extensive form of stochastic LP corresponding to the farmer's problem in Pyomo that accounts also for the price changes and solve it.



In [ ]:
model = pyo.ConcreteModel()

model.crops = pyo.Set(initialize=['W', 'C', 'S'])
model.totalacres = 500
model.factor_H = 1.2 # to obtain the yields in the good weather (high yield) case by multiplying the average ones
model.factor_L = 0.8 # to obtain the yields in the bad weather (low yield) case by multiplying the average ones
model.pricefactor_H = 0.9 # to obtain the prices in the good weather (high yield) case by multiplying the average ones
model.pricefactor_L = 1.1 # to obtain the prices in the bad weather (low yield) case by multiplying the average ones

# first stage variables
model.plant = pyo.Var(model.crops, within=pyo.NonNegativeReals) 

# first stage constraint
model.total_acres = pyo.Constraint(expr=pyo.summation(model.plant) <= model.totalacres)

model.scenarios = pyo.Set(initialize=['H', 'M', 'L'])  # high, medium, and low yield scenarios

# second stage variables (labelled as H,M,L depending on the scenario)
# the sell_extra variables refer to the amount of beets to be sold beyond the 6000 threshold, if any

model.sell_H = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_H = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_H = pyo.Var(within=pyo.NonNegativeReals) 

model.sell_M = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_M = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_M = pyo.Var(within=pyo.NonNegativeReals)

model.sell_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_L = pyo.Var(within=pyo.NonNegativeReals)

# second stage constraints
model.feed_cattle_W_H = pyo.Constraint(expr=model.plant['W'] * 2.5 * model.factor_H - model.sell_H['W'] + model.buy_H['W'] >= 200)
model.feed_cattle_C_H = pyo.Constraint(expr=model.plant['C'] * 3 * model.factor_H - model.sell_H['C'] + model.buy_H['C'] >= 240)
model.sell_S_extra_H = pyo.Constraint(expr=model.plant['S'] * 20 * model.factor_H >= model.sell_H['S'] + model.sell_extra_H)
model.sell_S_H = pyo.Constraint(expr=model.sell_H['S'] <= 6000)
model.nobuy_H = pyo.Constraint(expr=model.buy_H['S'] == 0)

model.feed_cattle_W_M = pyo.Constraint(expr=model.plant['W'] * 2.5 - model.sell_M['W'] + model.buy_M['W'] >= 200)
model.feed_cattle_C_M = pyo.Constraint(expr=model.plant['C'] * 3 - model.sell_M['C'] + model.buy_M['C'] >= 240)
model.sell_S_extra_M = pyo.Constraint(expr=model.plant['S'] * 20 >= model.sell_M['S'] + model.sell_extra_M)
model.sell_S_M = pyo.Constraint(expr=model.sell_M['S'] <= 6000)
model.nobuy_M = pyo.Constraint(expr=model.buy_M['S'] == 0)

model.feed_cattle_W_L = pyo.Constraint(expr=model.plant['W'] * 2.5 * model.factor_L - model.sell_L['W'] + model.buy_L['W'] >= 200)
model.feed_cattle_C_L = pyo.Constraint(expr=model.plant['C'] * 3 * model.factor_L - model.sell_L['C'] + model.buy_L['C'] >= 240)
model.sell_S_extra_L = pyo.Constraint(expr=model.plant['S'] * 20 * model.factor_L >= model.sell_L['S'] + model.sell_extra_L)
model.sell_S_L = pyo.Constraint(expr=model.sell_L['S'] <= 6000)
model.nobuy_L = pyo.Constraint(expr=model.buy_L['S'] == 0)

def first_stage_profit(model):
    return -model.plant["W"] * 150 - model.plant["C"] * 230 - model.plant["S"] * 260

model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

def second_stage_profit(model):
    total_H = -model.buy_H['W'] * 238 * model.pricefactor_H - model.buy_H['C'] * 210 * model.pricefactor_H + 36 * model.sell_H['S'] + 10 * model.sell_extra_H + model.sell_H['W'] * 170 * model.pricefactor_H + model.sell_H['C'] * 150 * model.pricefactor_H
    total_M = -model.buy_M['W'] * 238 - model.buy_M['C'] * 210 + 36 * model.sell_M['S'] + 10 * model.sell_extra_M + model.sell_M['W'] * 170 + model.sell_M['C'] * 150
    total_L = -model.buy_L['W'] * 238 * model.pricefactor_L - model.buy_L['C'] * 210 * model.pricefactor_L + 36 * model.sell_L['S'] + 10 * model.sell_extra_L + model.sell_L['W'] * 170 * model.pricefactor_L + model.sell_L['C'] * 150 * model.pricefactor_L
    return (total_H + total_M + total_L)/3.0

model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

def total_profit(model):
    return model.first_stage_profit + model.second_stage_profit

model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

result = cbc_solver.solve(model)
display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
display(Markdown(f"**Solution:**"))
display(Markdown(f"(land allocation) $x_1 = {model.plant['W'].value:.1f}$, $x_2 = {model.plant['C'].value:.1f}$, $x_3 = {model.plant['S'].value:.1f}$"))
display(Markdown(f"(recourse sell action high yield) $w_1 = {model.sell_H['W'].value:.1f}$, $w_2 = {model.sell_H['C'].value:.1f}$, $w_3 = {model.sell_H['S'].value:.1f}$, $w_4 = {model.sell_extra_H.value:.1f}$"))
display(Markdown(f"(recourse purchase action high yield) $y_1 = {model.buy_H['W'].value:.1f}$, $y_2 = {model.buy_H['C'].value:.1f}$, $y_3 = {model.buy_H['S'].value:.1f}$"))
display(Markdown(f"(recourse sell action medium yield) $w_1 = {model.sell_M['W'].value:.1f}$, $w_2 = {model.sell_M['C'].value:.1f}$, $w_3 = {model.sell_M['S'].value:.1f}$, $w_4 = {model.sell_extra_M.value:.1f}$"))
display(Markdown(f"(recourse purchase action medium yield) $y_1 = {model.buy_M['W'].value:.1f}$, $y_2 = {model.buy_M['C'].value:.1f}$, $y_3 = {model.buy_M['S'].value:.1f}$"))
display(Markdown(f"(recourse sell action low yield) $w_1 = {model.sell_L['W'].value:.1f}$, $w_2 = {model.sell_L['C'].value:.1f}$, $w_3 = {model.sell_L['S'].value:.1f}$, $w_4 = {model.sell_extra_L.value:.1f}$"))
display(Markdown(f"(recourse purchase action low yield) $y_1 = {model.buy_L['W'].value:.1f}$, $y_2 = {model.buy_L['C'].value:.1f}$, $y_3 = {model.buy_L['S'].value:.1f}$"))
display(Markdown(f"**Maximizes objective value to:** ${model.total_expected_profit():.0f}$€"))

**Solver status:** *ok, optimal*

**Solution:**

(land allocation) $x_1 = 170.0$, $x_2 = 80.0$, $x_3 = 250.0$

(recourse sell action high yield) $w_1 = 310.0$, $w_2 = 48.0$, $w_3 = 6000.0$, $w_4 = 0.0$

(recourse purchase action high yield) $y_1 = 0.0$, $y_2 = 0.0$, $y_3 = 0.0$

(recourse sell action medium yield) $w_1 = 225.0$, $w_2 = 0.0$, $w_3 = 5000.0$, $w_4 = 0.0$

(recourse purchase action medium yield) $y_1 = 0.0$, $y_2 = 0.0$, $y_3 = 0.0$

(recourse sell action low yield) $w_1 = 140.0$, $w_2 = 0.0$, $w_3 = 4000.0$, $w_4 = 0.0$

(recourse purchase action low yield) $y_1 = 0.0$, $y_2 = 48.0$, $y_3 = 0.0$

**Maximizes objective value to:** $106851$€

Consider again the case where prices are fixed and scenario-independent. The farmer possesses four fields of sizes $185$, $145$, $105$, and $65$ acres, respectively. (observe that the total of 500 acres is unchanged). For reasons of efficiency the farmer wants to raise only one type of crop on each fields. 

(c) Formulate this model as a two-stage stochastic program with a first-stage program with binary variables and solve it using once more the extensive form of the same stochastic program.

In [ ]:
model = pyo.ConcreteModel()

model.crops = pyo.Set(initialize=['W', 'C', 'S'])
model.fields = pyo.Set(initialize=['1','2','3','4'])
model.fieldsize = pyo.Param(model.fields, initialize={'1': 185.0, '2': 145.0, '3': 105.0, '4': 65.0}, within=pyo.NonNegativeReals)

model.factor_H = 1.2 # to obtain the yields in the good weather (high yield) case by multiplying the average ones
model.factor_L = 0.8 # to obtain the yields in the bad weather (low yield) case by multiplying the average ones
model.pricefactor_H = 1.0 # to obtain the prices in the good weather (high yield) case by multiplying the average ones
model.pricefactor_L = 1.0 # to obtain the prices in the bad weather (low yield) case by multiplying the average ones

# first stage variables, which are now binary variables
model.plant_W = pyo.Var(model.fields, within=pyo.Binary) 
model.plant_C = pyo.Var(model.fields, within=pyo.Binary) 
model.plant_S = pyo.Var(model.fields, within=pyo.Binary)

# first stage constraint
model.field1 = pyo.Constraint(expr=model.plant_W['1'] + model.plant_C['1'] + model.plant_S['1'] <= 1)
model.field2 = pyo.Constraint(expr=model.plant_W['2'] + model.plant_C['2'] + model.plant_S['2'] <= 1)
model.field3 = pyo.Constraint(expr=model.plant_W['3'] + model.plant_C['3'] + model.plant_S['3'] <= 1)
model.field4 = pyo.Constraint(expr=model.plant_W['4'] + model.plant_C['4'] + model.plant_S['4'] <= 1)

# we recalculate the total surface per type of crop
model.totalWsurface = pyo.Expression(expr=np.sum([model.plant_W[i] * model.fieldsize[i] for i in model.fields]))
model.totalCsurface = pyo.Expression(expr=np.sum([model.plant_C[i] * model.fieldsize[i] for i in model.fields]))
model.totalSsurface = pyo.Expression(expr=np.sum([model.plant_S[i] * model.fieldsize[i] for i in model.fields]))

model.scenarios = pyo.Set(initialize=['H', 'M', 'L'])  # high, medium, and low yield scenarios

# second stage variables (labelled as H,M,L depending on the scenario)
# the sell_extra variables refer to the amount of beets to be sold beyond the 6000 threshold, if any

model.sell_H = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_H = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_H = pyo.Var(within=pyo.NonNegativeReals) 

model.sell_M = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_M = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_M = pyo.Var(within=pyo.NonNegativeReals)

model.sell_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_L = pyo.Var(within=pyo.NonNegativeReals)

# second stage constraints
model.feed_cattle_W_H = pyo.Constraint(expr=model.totalWsurface * 2.5 * model.factor_H - model.sell_H['W'] + model.buy_H['W'] >= 200)
model.feed_cattle_C_H = pyo.Constraint(expr=model.totalCsurface * 3 * model.factor_H - model.sell_H['C'] + model.buy_H['C'] >= 240)
model.sell_S_extra_H = pyo.Constraint(expr=model.totalSsurface * 20 * model.factor_H >= model.sell_H['S'] + model.sell_extra_H)
model.sell_S_H = pyo.Constraint(expr=model.sell_H['S'] <= 6000)
model.nobuy_H = pyo.Constraint(expr=model.buy_H['S'] == 0)

model.feed_cattle_W_M = pyo.Constraint(expr=model.totalWsurface * 2.5 - model.sell_M['W'] + model.buy_M['W'] >= 200)
model.feed_cattle_C_M = pyo.Constraint(expr=model.totalCsurface * 3 - model.sell_M['C'] + model.buy_M['C'] >= 240)
model.sell_S_extra_M = pyo.Constraint(expr=model.totalSsurface * 20 >= model.sell_M['S'] + model.sell_extra_M)
model.sell_S_M = pyo.Constraint(expr=model.sell_M['S'] <= 6000)
model.nobuy_M = pyo.Constraint(expr=model.buy_M['S'] == 0)

model.feed_cattle_W_L = pyo.Constraint(expr=model.totalWsurface * 2.5 * model.factor_L - model.sell_L['W'] + model.buy_L['W'] >= 200)
model.feed_cattle_C_L = pyo.Constraint(expr=model.totalCsurface * 3 * model.factor_L - model.sell_L['C'] + model.buy_L['C'] >= 240)
model.sell_S_extra_L = pyo.Constraint(expr=model.totalSsurface * 20 * model.factor_L >= model.sell_L['S'] + model.sell_extra_L)
model.sell_S_L = pyo.Constraint(expr=model.sell_L['S'] <= 6000)
model.nobuy_L = pyo.Constraint(expr=model.buy_L['S'] == 0)

def first_stage_profit(model):
    return -model.totalWsurface * 150 - model.totalCsurface * 230 - model.totalSsurface * 260

model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

def second_stage_profit(model):
    total_H = -model.buy_H['W'] * 238 * model.pricefactor_H - model.buy_H['C'] * 210 * model.pricefactor_H + 36 * model.sell_H['S'] + 10 * model.sell_extra_H + model.sell_H['W'] * 170 * model.pricefactor_H + model.sell_H['C'] * 150 * model.pricefactor_H
    total_M = -model.buy_M['W'] * 238 - model.buy_M['C'] * 210 + 36 * model.sell_M['S'] + 10 * model.sell_extra_M + model.sell_M['W'] * 170 + model.sell_M['C'] * 150
    total_L = -model.buy_L['W'] * 238 * model.pricefactor_L - model.buy_L['C'] * 210 * model.pricefactor_L + 36 * model.sell_L['S'] + 10 * model.sell_extra_L + model.sell_L['W'] * 170 * model.pricefactor_L + model.sell_L['C'] * 150 * model.pricefactor_L
    return (total_H + total_M + total_L)/3.0

model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

def total_profit(model):
    return model.first_stage_profit + model.second_stage_profit

model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

result = cbc_solver.solve(model)
display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
display(Markdown(f"**Solution:**"))
display(Markdown(f"(wheat field allocation) $[{model.plant_W['1'].value:.0f}, {model.plant_W['2'].value:.0f}, {model.plant_W['3'].value:.0f}, {model.plant_W['4'].value:.0f}]$"))
display(Markdown(f"(corn field allocation) $[{model.plant_C['1'].value:.0f}, {model.plant_C['2'].value:.0f}, {model.plant_C['3'].value:.0f}, {model.plant_C['4'].value:.0f}]$"))
display(Markdown(f"(beet field allocation) $[{model.plant_S['1'].value:.0f}, {model.plant_S['2'].value:.0f}, {model.plant_S['3'].value:.0f}, {model.plant_S['4'].value:.0f}]$"))
display(Markdown(f"(land field allocation) $x_1 = {model.totalWsurface.expr():.1f}$, $x_2 = {model.totalCsurface.expr():.1f}$, $x_3 = {model.totalSsurface.expr():.1f}$"))
display(Markdown(f"(recourse sell action high yield) $w_1 = {model.sell_H['W'].value:.1f}$, $w_2 = {model.sell_H['C'].value:.1f}$, $w_3 = {model.sell_H['S'].value:.1f}$, $w_4 = {model.sell_extra_H.value:.1f}$"))
display(Markdown(f"(recourse purchase action high yield) $y_1 = {model.buy_H['W'].value:.1f}$, $y_2 = {model.buy_H['C'].value:.1f}$, $y_3 = {model.buy_H['S'].value:.1f}$"))
display(Markdown(f"(recourse sell action medium yield) $w_1 = {model.sell_M['W'].value:.1f}$, $w_2 = {model.sell_M['C'].value:.1f}$, $w_3 = {model.sell_M['S'].value:.1f}$, $w_4 = {model.sell_extra_M.value:.1f}$"))
display(Markdown(f"(recourse purchase action medium yield) $y_1 = {model.buy_M['W'].value:.1f}$, $y_2 = {model.buy_M['C'].value:.1f}$, $y_3 = {model.buy_M['S'].value:.1f}$"))
display(Markdown(f"(recourse sell action low yield) $w_1 = {model.sell_L['W'].value:.1f}$, $w_2 = {model.sell_L['C'].value:.1f}$, $w_3 = {model.sell_L['S'].value:.1f}$, $w_4 = {model.sell_extra_L.value:.1f}$"))
display(Markdown(f"(recourse purchase action low yield) $y_1 = {model.buy_L['W'].value:.1f}$, $y_2 = {model.buy_L['C'].value:.1f}$, $y_3 = {model.buy_L['S'].value:.1f}$"))
display(Markdown(f"**Maximizes objective value to:** ${model.total_expected_profit():.0f}$€"))

**Solver status:** *ok, optimal*

**Solution:**

(wheat field allocation) $[0, 1, 0, 0]$

(corn field allocation) $[0, 0, 1, 0]$

(beet field allocation) $[1, 0, 0, 1]$

...

(recourse sell action medium yield) $w_1 = 162.5$, $w_2 = 75.0$, $w_3 = 5000.0$, $w_4 = 0.0$

(recourse purchase action medium yield) $y_1 = 0.0$, $y_2 = 0.0$, $y_3 = 0.0$

(recourse sell action low yield) $w_1 = 90.0$, $w_2 = 12.0$, $w_3 = 4000.0$, $w_4 = 0.0$

(recourse purchase action low yield) $y_1 = 0.0$, $y_2 = 0.0$, $y_3 = 0.0$

**Maximizes objective value to:** $107975$€

Consider again the setting described in (a). The farmer would normally act as a risk-averse person and simply plan for the worst case. More precisely, the farmer maximizes her profit under the worst scenario, that is the bad weather one.

(d) Solve the deterministic LP for the bad weather scenario to find the corresponding worst-case optimal land allocation. Using this land allocation, compute the loss in expected profit if that solution is taken.

In [ ]:
# Bad weather only
model = pyo.ConcreteModel()

model.crops = pyo.Set(initialize=['W', 'C', 'S'])
model.totalacres = 500
model.factor_H = 1.2 # to obtain the yields in the good weather (high yield) case by multiplying the average ones
model.factor_L = 0.8 # to obtain the yields in the bad weather (low yield) case by multiplying the average ones

# first stage variables
model.plant = pyo.Var(model.crops, within=pyo.NonNegativeReals) 

# first stage constraint
model.total_acres = pyo.Constraint(expr=pyo.summation(model.plant) <= model.totalacres)

# second stage variables
model.sell_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_L = pyo.Var(within=pyo.NonNegativeReals)

# second stage constraints
model.feed_cattle_W_L = pyo.Constraint(expr=model.plant['W'] * 2.5 * model.factor_L - model.sell_L['W'] + model.buy_L['W'] >= 200)
model.feed_cattle_C_L = pyo.Constraint(expr=model.plant['C'] * 3 * model.factor_L - model.sell_L['C'] + model.buy_L['C'] >= 240)
model.sell_S_extra_L = pyo.Constraint(expr=model.plant['S'] * 20 * model.factor_L >= model.sell_L['S'] + model.sell_extra_L)
model.sell_S_L = pyo.Constraint(expr=model.sell_L['S'] <= 6000)
model.nobuy_L = pyo.Constraint(expr=model.buy_L['S'] == 0)

def first_stage_profit(model):
    return -model.plant["W"] * 150 - model.plant["C"] * 230 - model.plant["S"] * 260

model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

def second_stage_profit(model):
    return -model.buy_L['W'] * 238 - model.buy_L['C'] * 210 + 36 * model.sell_L['S'] + 10 * model.sell_extra_L + model.sell_L['W'] * 170 + model.sell_L['C'] * 150

model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

def total_profit(model):
    return model.first_stage_profit + model.second_stage_profit

model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

result = cbc_solver.solve(model)
display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
display(Markdown(f"**Solution:**"))
display(Markdown(f"(land allocation) $x_1 = {model.plant['W'].value:.1f}$, $x_2 = {model.plant['C'].value:.1f}$, $x_3 = {model.plant['S'].value:.1f}$"))
display(Markdown(f"(recourse sell action low yield) $w_1 = {model.sell_L['W'].value:.1f}$, $w_2 = {model.sell_L['C'].value:.1f}$, $w_3 = {model.sell_L['S'].value:.1f}$, $w_4 = {model.sell_extra_L.value:.1f}$"))
display(Markdown(f"(recourse purchase action low yield) $y_1 = {model.buy_L['W'].value:.1f}$, $y_2 = {model.buy_L['C'].value:.1f}$, $y_3 = {model.buy_L['S'].value:.1f}$"))
display(Markdown(f"**Maximizes objective value to:** ${model.total_expected_profit():.0f}$€"))

**Solver status:** *ok, optimal*

**Solution:**

(land allocation) $x_1 = 100.0$, $x_2 = 25.0$, $x_3 = 375.0$

(recourse sell action low yield) $w_1 = 0.0$, $w_2 = 0.0$, $w_3 = 6000.0$, $w_4 = 0.0$

(recourse purchase action low yield) $y_1 = 0.0$, $y_2 = 180.0$, $y_3 = 0.0$

**Maximizes objective value to:** $59950$€

In [ ]:
# Expected profit when optimizing only against bad weather 
model = pyo.ConcreteModel()

model.crops = pyo.Set(initialize=['W', 'C', 'S'])
model.factor_H = 1.2 # to obtain the yields in the good weather (high yield) case by multiplying the average ones
model.factor_L = 0.8 # to obtain the yields in the bad weather (low yield) case by multiplying the average ones

# first stage variables are now parameters, since we set them equal to the optimal allocation for bad weather calculated in the previous cell
model.plant = pyo.Param(model.crops, within=pyo.NonNegativeReals, initialize={'W': 100, 'C': 25, 'S': 375}) 

model.scenarios = pyo.Set(initialize=['H', 'M', 'L'])  # high, medium, and low yield scenarios

# second stage variables (labelled as H,M,L depending on the scenario)
# the sell_extra variables refer to the amount of beets to be sold beyond the 6000 threshold, if any
model.sell_H = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_H = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_H = pyo.Var(within=pyo.NonNegativeReals) 

model.sell_M = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_M = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_M = pyo.Var(within=pyo.NonNegativeReals)

model.sell_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.buy_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
model.sell_extra_L = pyo.Var(within=pyo.NonNegativeReals)

# second stage constraints
model.feed_cattle_W_H = pyo.Constraint(expr=model.plant['W'] * 2.5 * model.factor_H - model.sell_H['W'] + model.buy_H['W'] >= 200)
model.feed_cattle_C_H = pyo.Constraint(expr=model.plant['C'] * 3 * model.factor_H - model.sell_H['C'] + model.buy_H['C'] >= 240)
model.sell_S_extra_H = pyo.Constraint(expr=model.plant['S'] * 20 * model.factor_H >= model.sell_H['S'] + model.sell_extra_H)
model.sell_S_H = pyo.Constraint(expr=model.sell_H['S'] <= 6000)
model.nobuy_H = pyo.Constraint(expr=model.buy_H['S'] == 0)

model.feed_cattle_W_M = pyo.Constraint(expr=model.plant['W'] * 2.5 - model.sell_M['W'] + model.buy_M['W'] >= 200)
model.feed_cattle_C_M = pyo.Constraint(expr=model.plant['C'] * 3 - model.sell_M['C'] + model.buy_M['C'] >= 240)
model.sell_S_extra_M = pyo.Constraint(expr=model.plant['S'] * 20 >= model.sell_M['S'] + model.sell_extra_M)
model.sell_S_M = pyo.Constraint(expr=model.sell_M['S'] <= 6000)
model.nobuy_M = pyo.Constraint(expr=model.buy_M['S'] == 0)

model.feed_cattle_W_L = pyo.Constraint(expr=model.plant['W'] * 2.5 * model.factor_L - model.sell_L['W'] + model.buy_L['W'] >= 200)
model.feed_cattle_C_L = pyo.Constraint(expr=model.plant['C'] * 3 * model.factor_L - model.sell_L['C'] + model.buy_L['C'] >= 240)
model.sell_S_extra_L = pyo.Constraint(expr=model.plant['S'] * 20 * model.factor_L >= model.sell_L['S'] + model.sell_extra_L)
model.sell_S_L = pyo.Constraint(expr=model.sell_L['S'] <= 6000)
model.nobuy_L = pyo.Constraint(expr=model.buy_L['S'] == 0)

def first_stage_profit(model):
    return -model.plant["W"] * 150 - model.plant["C"] * 230 - model.plant["S"] * 260

model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

def second_stage_profit(model):
    total_H = -model.buy_H['W'] * 238 - model.buy_H['C'] * 210 + 36 * model.sell_H['S'] + 10 * model.sell_extra_H + model.sell_H['W'] * 170 + model.sell_H['C'] * 150
    total_M = -model.buy_M['W'] * 238 - model.buy_M['C'] * 210 + 36 * model.sell_M['S'] + 10 * model.sell_extra_M + model.sell_M['W'] * 170 + model.sell_M['C'] * 150
    total_L = -model.buy_L['W'] * 238 - model.buy_L['C'] * 210 + 36 * model.sell_L['S'] + 10 * model.sell_extra_L + model.sell_L['W'] * 170 + model.sell_L['C'] * 150
    return (total_H + total_M + total_L)/3.0

model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

def total_profit(model):
    return model.first_stage_profit + model.second_stage_profit

model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

result = cbc_solver.solve(model)
display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
display(Markdown(f"**Solution:**"))
display(Markdown(f"(land allocation) $x_1 = {model.plant['W']:.1f}$, $x_2 = {model.plant['C']:.1f}$, $x_3 = {model.plant['S']:.1f}$"))
display(Markdown(f"(recourse sell action high yield) $w_1 = {model.sell_H['W'].value:.1f}$, $w_2 = {model.sell_H['C'].value:.1f}$, $w_3 = {model.sell_H['S'].value:.1f}$, $w_4 = {model.sell_extra_H.value:.1f}$"))
display(Markdown(f"(recourse purchase action high yield) $y_1 = {model.buy_H['W'].value:.1f}$, $y_2 = {model.buy_H['C'].value:.1f}$, $y_3 = {model.buy_H['S'].value:.1f}$"))
display(Markdown(f"(recourse sell action medium yield) $w_1 = {model.sell_M['W'].value:.1f}$, $w_2 = {model.sell_M['C'].value:.1f}$, $w_3 = {model.sell_M['S'].value:.1f}$, $w_4 = {model.sell_extra_M.value:.1f}$"))
display(Markdown(f"(recourse purchase action medium yield) $y_1 = {model.buy_M['W'].value:.1f}$, $y_2 = {model.buy_M['C'].value:.1f}$, $y_3 = {model.buy_M['S'].value:.1f}$"))
display(Markdown(f"(recourse sell action low yield) $w_1 = {model.sell_L['W'].value:.1f}$, $w_2 = {model.sell_L['C'].value:.1f}$, $w_3 = {model.sell_L['S'].value:.1f}$, $w_4 = {model.sell_extra_L.value:.1f}$"))
display(Markdown(f"(recourse purchase action low yield) $y_1 = {model.buy_L['W'].value:.1f}$, $y_2 = {model.buy_L['C'].value:.1f}$, $y_3 = {model.buy_L['S'].value:.1f}$"))
display(Markdown(f"**Maximizes objective value to:** ${model.total_expected_profit():.0f}$€"))

**Solver status:** *ok, optimal*

**Solution:**

(land allocation) $x_1 = 100.0$, $x_2 = 25.0$, $x_3 = 375.0$

(recourse sell action high yield) $w_1 = 100.0$, $w_2 = 0.0$, $w_3 = 6000.0$, $w_4 = 3000.0$

(recourse purchase action high yield) $y_1 = 0.0$, $y_2 = 150.0$, $y_3 = 0.0$

(recourse sell action medium yield) $w_1 = 50.0$, $w_2 = 0.0$, $w_3 = 6000.0$, $w_4 = 1500.0$

(recourse purchase action medium yield) $y_1 = 0.0$, $y_2 = 165.0$, $y_3 = 0.0$

(recourse sell action low yield) $w_1 = 0.0$, $w_2 = 0.0$, $w_3 = 6000.0$, $w_4 = 0.0$

(recourse purchase action low yield) $y_1 = 0.0$, $y_2 = 180.0$, $y_3 = 0.0$

**Maximizes objective value to:** $86600$€

(e) A different approach situation be to require a reasonable minimum profit under the worst case. Find the solution that maximizes the expected profit under the constraint that in the worst case the profit does not fall below $58.000$ euro. What is now the loss in expected profit?

Repeat the same optimization also with other values of minimal profit: $56.000$, $54.000$, $52.000$, $50.000$, and $48.000$ euro. Graph the curve of expected profit loss and compare the associated optimal decisions.

In [ ]:
expectedprofit_noconstraints = 108390

def FarmersWithMinimumProfit(minprofit):
    model = pyo.ConcreteModel()

    model.crops = pyo.Set(initialize=['W', 'C', 'S'])
    model.totalacres = 500
    model.factor_H = 1.2 # to obtain the yields in the good weather (high yield) case by multiplying the average ones
    model.factor_L = 0.8 # to obtain the yields in the bad weather (low yield) case by multiplying the average ones
    model.pricefactor_H = 1.0 # to obtain the prices in the good weather (high yield) case by multiplying the average ones
    model.pricefactor_L = 1.0 # to obtain the prices in the bad weather (low yield) case by multiplying the average ones

    # first stage variables
    model.plant = pyo.Var(model.crops, within=pyo.NonNegativeReals) 

    # first stage constraint
    model.total_acres = pyo.Constraint(expr=pyo.summation(model.plant) <= model.totalacres)

    model.scenarios = pyo.Set(initialize=['H', 'M', 'L'])  # high, medium, and low yield scenarios

    # second stage variables (labelled as H,M,L depending on the scenario)
    # the sell_extra variables refer to the amount of beets to be sold beyond the 6000 threshold, if any
    model.sell_H = pyo.Var(model.crops, within=pyo.NonNegativeReals)
    model.buy_H = pyo.Var(model.crops, within=pyo.NonNegativeReals)
    model.sell_extra_H = pyo.Var(within=pyo.NonNegativeReals) 

    model.sell_M = pyo.Var(model.crops, within=pyo.NonNegativeReals)
    model.buy_M = pyo.Var(model.crops, within=pyo.NonNegativeReals)
    model.sell_extra_M = pyo.Var(within=pyo.NonNegativeReals)

    model.sell_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
    model.buy_L = pyo.Var(model.crops, within=pyo.NonNegativeReals)
    model.sell_extra_L = pyo.Var(within=pyo.NonNegativeReals)

    # second stage constraints
    model.feed_cattle_W_H = pyo.Constraint(expr=model.plant['W'] * 2.5 * model.factor_H - model.sell_H['W'] + model.buy_H['W'] >= 200)
    model.feed_cattle_C_H = pyo.Constraint(expr=model.plant['C'] * 3 * model.factor_H - model.sell_H['C'] + model.buy_H['C'] >= 240)
    model.sell_S_extra_H = pyo.Constraint(expr=model.plant['S'] * 20 * model.factor_H >= model.sell_H['S'] + model.sell_extra_H)
    model.sell_S_H = pyo.Constraint(expr=model.sell_H['S'] <= 6000)
    model.nobuy_H = pyo.Constraint(expr=model.buy_H['S'] == 0)

    model.feed_cattle_W_M = pyo.Constraint(expr=model.plant['W'] * 2.5 - model.sell_M['W'] + model.buy_M['W'] >= 200)
    model.feed_cattle_C_M = pyo.Constraint(expr=model.plant['C'] * 3 - model.sell_M['C'] + model.buy_M['C'] >= 240)
    model.sell_S_extra_M = pyo.Constraint(expr=model.plant['S'] * 20 >= model.sell_M['S'] + model.sell_extra_M)
    model.sell_S_M = pyo.Constraint(expr=model.sell_M['S'] <= 6000)
    model.nobuy_M = pyo.Constraint(expr=model.buy_M['S'] == 0)

    model.feed_cattle_W_L = pyo.Constraint(expr=model.plant['W'] * 2.5 * model.factor_L - model.sell_L['W'] + model.buy_L['W'] >= 200)
    model.feed_cattle_C_L = pyo.Constraint(expr=model.plant['C'] * 3 * model.factor_L - model.sell_L['C'] + model.buy_L['C'] >= 240)
    model.sell_S_extra_L = pyo.Constraint(expr=model.plant['S'] * 20 * model.factor_L >= model.sell_L['S'] + model.sell_extra_L)
    model.sell_S_L = pyo.Constraint(expr=model.sell_L['S'] <= 6000)
    model.nobuy_L = pyo.Constraint(expr=model.buy_L['S'] == 0)

    def first_stage_profit(model):
        return -model.plant["W"] * 150 - model.plant["C"] * 230 - model.plant["S"] * 260

    model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

    def second_stage_profit(model):
        total_H = -model.buy_H['W'] * 238 * model.pricefactor_H - model.buy_H['C'] * 210 * model.pricefactor_H + 36 * model.sell_H['S'] + 10 * model.sell_extra_H + model.sell_H['W'] * 170 * model.pricefactor_H + model.sell_H['C'] * 150 * model.pricefactor_H
        total_M = -model.buy_M['W'] * 238 - model.buy_M['C'] * 210 + 36 * model.sell_M['S'] + 10 * model.sell_extra_M + model.sell_M['W'] * 170 + model.sell_M['C'] * 150
        total_L = -model.buy_L['W'] * 238 * model.pricefactor_L - model.buy_L['C'] * 210 * model.pricefactor_L + 36 * model.sell_L['S'] + 10 * model.sell_extra_L + model.sell_L['W'] * 170 * model.pricefactor_L + model.sell_L['C'] * 150 * model.pricefactor_L
        return (total_H + total_M + total_L)/3.0

    model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

    def total_profit(model):
        return model.first_stage_profit + model.second_stage_profit

    model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

    model.totalprofit_H = pyo.Expression(expr = -model.plant["W"] * 150 - model.plant["C"] * 230 - model.plant["S"] * 260 -model.buy_H['W'] * 238 * model.pricefactor_H - model.buy_H['C'] * 210 * model.pricefactor_H + 36 * model.sell_H['S'] + 10 * model.sell_extra_H + model.sell_H['W'] * 170 * model.pricefactor_H + model.sell_H['C'] * 150 * model.pricefactor_H)
    model.totalprofit_M = pyo.Expression(expr = -model.plant["W"] * 150 - model.plant["C"] * 230 - model.plant["S"] * 260 -model.buy_M['W'] * 238 - model.buy_M['C'] * 210 + 36 * model.sell_M['S'] + 10 * model.sell_extra_M + model.sell_M['W'] * 170 + model.sell_M['C'] * 150)
    model.totalprofit_L = pyo.Expression(expr = -model.plant["W"] * 150 - model.plant["C"] * 230 - model.plant["S"] * 260 -model.buy_L['W'] * 238 * model.pricefactor_L - model.buy_L['C'] * 210 * model.pricefactor_L + 36 * model.sell_L['S'] + 10 * model.sell_extra_L + model.sell_L['W'] * 170 * model.pricefactor_L + model.sell_L['C'] * 150 * model.pricefactor_L)
    model.minimum_profit = pyo.Constraint(expr=model.totalprofit_L >= minprofit)
    result = cbc_solver.solve(model)

    display(Markdown(f"**Minimum profit threshold** of ${minprofit:.0f}$€ leads to an **optimal expected profit** of ${model.total_expected_profit():.0f}$€, with a **loss** of ${expectedprofit_noconstraints - model.total_expected_profit():.0f}$€"))
    return model.total_expected_profit()

profitthresholds = [48000,50000,52000,54000,56000,58000]
for minprofit in profitthresholds:
    FarmersWithMinimumProfit(minprofit)

**Minimum profit threshold** of $48000$€ leads to an **optimal expected profit** of $108390$€, with a **loss** of $0$€

**Minimum profit threshold** of $50000$€ leads to an **optimal expected profit** of $108292$€, with a **loss** of $98$€

**Minimum profit threshold** of $52000$€ leads to an **optimal expected profit** of $107976$€, with a **loss** of $414$€

**Minimum profit threshold** of $54000$€ leads to an **optimal expected profit** of $107611$€, with a **loss** of $779$€

**Minimum profit threshold** of $56000$€ leads to an **optimal expected profit** of $107246$€, with a **loss** of $1144$€

**Minimum profit threshold** of $58000$€ leads to an **optimal expected profit** of $101176$€, with a **loss** of $7214$€

# Exercise 2: News vendor problem 

Consider the news vendor problem seen at lecture and suppose that. Suppose the unit prices are $c = 10 , s = 25 , r = 5$, and that the newspaper demand $\xi$ modelled as a continuous random variable.

(a) Find the optimal solution of the news vendor problem using the explicit formula that features the inverse CDFs/quantile functions for the following three distributions:
 - a uniform distribution on the interval $[50,150]$;
*Hint: see [Uniform distribution CDF and its inverse](https://en.wikipedia.org/wiki/Continuous_uniform_distribution#Cumulative_distribution_function)*
 - a Pareto distribution on the interval $[50,+\infty)$ with $x_m=50$ and exponent $\alpha=2$. *Hint: the inverse CDF for a Pareto distribution is given by* $H^{-1}(\varepsilon) = \frac{x_m}{(1-\varepsilon)^{1/\alpha}}$.
 - a Weibull distribution on the interval $[0,+\infty)$ with shape parameter $k=2$ and scale parameter $\lambda=113$, see [Weibull distribution CDF and its inverse](https://en.wikipedia.org/wiki/Weibull_distribution#Cumulative_distribution_function).


In [ ]:
# Setting the parameters
c = 10
s = 25
r = 5

# Definining the three inverse CDFs/quantile functions for the three distributions
def quantileuniform(epsilon, a, b):
  return a + epsilon*(b-a)

def quantilepareto(epsilon, xm, alpha):
  return xm/(1.0-epsilon)**(1.0/alpha)

def quantileweibull(epsilon, k, l):
  return l*(-np.log(1-epsilon))**(1.0/k)

# Calculating the optimal decision for each of the three distributions
display(Markdown(f"**Optimal solution** for uniform distribution: ${quantileuniform((s-c)/(s-r),50,150):.2f}$"))
display(Markdown(f"**Optimal solution** for Pareto distribution: ${quantilepareto((s-c)/(s-r),50,2):.2f}$"))
display(Markdown(f"**Optimal solution** for Weibull distribution: ${quantileweibull((s-c)/(s-r),2,113):.2f}$"))

**Optimal solution** for uniform distribution: $125.00$

**Optimal solution** for Pareto distribution: $100.00$

**Optimal solution** for Weibull distribution: $133.05$

Note that all the three distribution above have the same expected value, that is $\mathbb E \xi = 100$.

(b) Find the optimal solution of the deterministic LP model obtained by assuming the demand is fixed $\xi=\bar{\xi}$ and equal to the average demand $\bar{\xi} = \mathbb E \xi = 100$.

In [ ]:
# Two-stage stochastic LP

c = 10
s = 25
r = 5

model = pyo.ConcreteModel()
model.xi = 100

# first stage variable
model.x = pyo.Var(within=pyo.NonNegativeReals) #bought

def first_stage_profit(model):
    return -c * model.x

model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

# second stage variables
model.y = pyo.Var(within=pyo.NonNegativeReals) #sold
model.z = pyo.Var(within=pyo.NonNegativeReals) #unsold to be returned 

# second stage constraints
model.cantsoldthingsidonthave = pyo.Constraint(expr=model.y <= model.xi)
model.newspapersdonotdisappear = pyo.Constraint(expr=model.y + model.z == model.x)

def second_stage_profit(model):
    return s * model.y + r * model.z

model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

def total_profit(model):
    return model.first_stage_profit + model.second_stage_profit

model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

result = cbc_solver.solve(model)

display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
display(Markdown(f"**Optimal solution** for determistic demand equal to $100$: $x = {model.x.value:.1f}$"))
display(Markdown(f"**Optimal deterministic profit:** ${model.total_expected_profit():.0f}$€"))

**Solver status:** *ok, optimal*

**Optimal solution** for determistic demand equal to $100$: $x = 100.0$

**Optimal deterministic profit:** $1500$€

We now assess how well we perform taking the average demand as optimal decision the news vendor problem for each of the three demand distributions above.

(c) For a fixed decision variable $x=100$, approximate the expected profit of the news vendor for each of the three distributions above using the Sample Average Approximation method with $N=2500$ points. More specifically, generate $N=2500$ samples from the considered distribution and solve the extensive form of the stochastic LP resulting from those $N=2500$ scenarios.

In [ ]:
# SAA of the two-stage stochastic LP to calculate the expected profit when buying the average

def NaiveNewsvendorSAA(N, sample, distributiontype):

    model = pyo.ConcreteModel()

    def indices_rule(model):
        return range(N)
    model.indices = pyo.Set(initialize=indices_rule)
    model.xi = pyo.Param(model.indices, initialize=dict(enumerate(sample)))

    # first stage variable
    model.x = 100.0 #bought

    def first_stage_profit(model):
        return -c * model.x

    model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

    # second stage variables
    model.y = pyo.Var(model.indices, within=pyo.NonNegativeReals) #sold
    model.z = pyo.Var(model.indices, within=pyo.NonNegativeReals) #unsold to be returned 

    # second stage constraints
    model.cantsoldthingsidonthave = pyo.ConstraintList()
    model.newspapersdonotdisappear = pyo.ConstraintList()
    for i in model.indices:
        model.cantsoldthingsidonthave.add(expr=model.y[i] <= model.xi[i])
        model.newspapersdonotdisappear.add(expr=model.y[i] + model.z[i] == model.x)

    def second_stage_profit(model):
        return sum([s * model.y[i] + r * model.z[i] for i in model.indices])/float(N)

    model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

    def total_profit(model):
        return model.first_stage_profit + model.second_stage_profit

    model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

    result = cbc_solver.solve(model)

    display(Markdown(f"**Approximate expected optimal profit when using the average** $x=100$ with {distributiontype} demand: ${model.total_expected_profit():.2f}$€"))
    return model.total_expected_profit()

np.random.seed(20122020)
N = 2500

samples = np.random.uniform(low=50.0, high=150.0, size=N)
naiveprofit_uniform = NaiveNewsvendorSAA(N, samples, 'uniform')

shape = 2
xm = 50
samples = (np.random.pareto(a=shape, size=N) + 1) *  xm
naiveprofit_pareto = NaiveNewsvendorSAA(N, samples, 'Pareto')

shape=2
scale=113
samples = scale*np.random.weibull(a=shape, size=N)
naiveprofit_weibull = NaiveNewsvendorSAA(N, samples, 'Weibull')

**Approximate expected optimal profit when using the average** $x=100$ with uniform demand: $1245.56$€

**Approximate expected optimal profit when using the average** $x=100$ with Pareto demand: $1009.53$€

**Approximate expected optimal profit when using the average** $x=100$ with Weibull demand: $1088.44$€

(d) Solve approximately the news vendor problem for each of the three distributions above using the Sample Average Approximation method with $N=2500$ points. More specifically, generate $N=2500$ samples from the considered distribution and solve the extensive form of the stochastic LP resulting from those $N=2500$ scenarios. For each of the three distribution, compare the optimal expected profit with that obtained in (c) and calculate the value of the stochastic solution (VSS).

In [ ]:
# Two-stage stochastic LP for uniform distribution

c = 10
s = 25
r = 5

def NewsvendorSAA(N, sample, distributiontype):

    model = pyo.ConcreteModel()

    def indices_rule(model):
        return range(N)
    model.indices = pyo.Set(initialize=indices_rule)
    model.xi = pyo.Param(model.indices, initialize=dict(enumerate(sample)))

    # first stage variable
    model.x = pyo.Var(within=pyo.NonNegativeReals) #bought

    def first_stage_profit(model):
        return -c * model.x

    model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

    # second stage variables
    model.y = pyo.Var(model.indices, within=pyo.NonNegativeReals) #sold
    model.z = pyo.Var(model.indices, within=pyo.NonNegativeReals) #unsold to be returned 

    # second stage constraints
    model.cantsoldthingsidonthave = pyo.ConstraintList()
    model.newspapersdonotdisappear = pyo.ConstraintList()
    for i in model.indices:
        model.cantsoldthingsidonthave.add(expr=model.y[i] <= model.xi[i])
        model.newspapersdonotdisappear.add(expr=model.y[i] + model.z[i] == model.x)

    def second_stage_profit(model):
        return sum([s * model.y[i] + r * model.z[i] for i in model.indices])/float(N)

    model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

    def total_profit(model):
        return model.first_stage_profit + model.second_stage_profit

    model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

    result = cbc_solver.solve(model)

    display(Markdown(f"**Approximate news vendor problem solution for:** {distributiontype} distribution **using:** $N={N:.0f}$ points"))
    display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
    display(Markdown(f"**Approximate optimal solution:** $x = {model.x.value:.2f}$"))
    display(Markdown(f"**Approximate expected optimal profit:** ${model.total_expected_profit():.2f}$€"))
    return model.total_expected_profit()

np.random.seed(20122020)
N = 2500

samples = np.random.uniform(low=50.0, high=150.0, size=N)
smartprofit_uniform = NewsvendorSAA(N, samples, 'uniform')
display(Markdown(f"**Value of the stochastic solution:** ${smartprofit_uniform:.2f}-{naiveprofit_uniform:.2f} = {smartprofit_uniform-naiveprofit_uniform:.2f}$€"))

shape = 2
xm = 50
samples = (np.random.pareto(a=shape, size=N) + 1) *  xm
smartprofit_pareto = NewsvendorSAA(N, samples, 'Pareto')
display(Markdown(f"**Value of the stochastic solution:** ${smartprofit_pareto:.2f}-{naiveprofit_pareto:.2f} = {smartprofit_pareto-naiveprofit_pareto:.2f}$€"))

shape=2
scale=113
samples = scale*np.random.weibull(a=shape, size=N)
smartprofit_weibull =NewsvendorSAA(N, samples, 'Weibull')
display(Markdown(f"**Value of the stochastic solution:** ${smartprofit_weibull:.2f}-{naiveprofit_weibull:.2f} = {smartprofit_weibull-naiveprofit_weibull:.2f}$€"))

**Approximate news vendor problem solution for:** uniform distribution **using:** $N=2500$ points

**Solver status:** *ok, optimal*

**Approximate optimal solution:** $x = 123.94$

**Approximate expected optimal profit:** $1302.12$€

**Value of the stochastic solution:** $1302.12-1245.56 = 56.56$€

...

**Approximate news vendor problem solution for:** Weibull distribution **using:** $N=2500$ points

**Solver status:** *ok, optimal*

**Approximate optimal solution:** $x = 133.15$

**Approximate expected optimal profit:** $1157.84$€

**Value of the stochastic solution:** $1157.84-1088.44 = 69.40$€

# Exercise 3: Airline seat allocation problem

**This is the same problem as in Exercise 3 of the Week 4 tutorial exercise sheet (use its solutions)**

An airlines is trying to decide how to partition a new plane for the Amsterdam-Buenos Aires route. This plane can seat 200 economy class passengers. A section can be created for first class seats but each of these seats takes the space of 2 economy class seats. A business class section can also be created, but each of these seats takes as much space as 1.5 economy class seats. The profit on a first class ticket is, however, three times the profit of an economy ticket, while a business class ticket has a profit of two times an economy ticket's profit. Once the plane is partitioned into these seating classes, it cannot be changed. The airlines knows, however, that the plane will not always be full in each section. They have decided that three scenarios will occur with about the same frequency: 

(1) weekday morning and evening traffic, 

(2) weekend traffic, 

(3) weekday midday traffic. 

Under Scenario 1, they think they can sell as many as 20 first class tickets, 50 business class tickets, and 200 economy tickets. Under Scenario 2, these figures are $10 , 25 $, and $175$, while under Scenario 3, they are $5 , 10$, and $150$. The following table reports the forecast demand in the three scenarios.

| Scenario | First class seats | Business class seats | Economy class seats |
| :-: | :-: | :-: | :-: |
| Scenario 1 | 20 | 50 | 200 |
| Scenario 2 | 10 | 25 | 175 |
| Scenario 3 | 5 | 10 | 150 |

Despite these estimates, the airline will not sell more tickets than seats in each of the sections (hence no overbooking strategy).

(a) Implement and solve the extensive form of the stochastic program for the optimal seat allocation aiming to maximize the airline profit.

In [ ]:
model = pyo.ConcreteModel()

model.classes = pyo.Set(initialize=['F', 'B', 'E'])
model.totalseats = 200
model.pricefactor_F = 3.0
model.pricefactor_B = 2.0
model.seatfactor_F = 2.0 
model.seatfactor_B = 1.5

# first stage variables
model.seats = pyo.Var(model.classes, within=pyo.NonNegativeIntegers) 

# first stage constraint
model.equivalentseatsF = pyo.Expression(expr=model.seats['F']*model.seatfactor_F)
model.equivalentseatsB = pyo.Expression(expr=model.seats['B']*model.seatfactor_B)
model.equivalentseatsE = pyo.Expression(expr=model.seats['E'])
model.planeseats = pyo.Constraint(expr=model.equivalentseatsF + model.equivalentseatsB + model.equivalentseatsE <= model.totalseats)

model.scenarios = pyo.Set(initialize=[1,2,3])

# second stage variables (labelled as 1,2,3 depending on the scenario)
model.sell = pyo.Var(model.classes, model.scenarios, within=pyo.NonNegativeIntegers)

# second stage constraints
model.demandF_1 = pyo.Constraint(expr= model.sell['F',1] <= 20)
model.limitF_1 = pyo.Constraint(expr= model.sell['F',1] <= model.seats['F'])
model.demandF_2 = pyo.Constraint(expr= model.sell['F',2] <= 10)
model.limitF_2 = pyo.Constraint(expr= model.sell['F',2] <= model.seats['F'])
model.demandF_3 = pyo.Constraint(expr= model.sell['F',3] <= 5)
model.limitF_3 = pyo.Constraint(expr= model.sell['F',3] <= model.seats['F'])
model.demandB_1 = pyo.Constraint(expr= model.sell['B',1] <= 50)
model.limitB_1 = pyo.Constraint(expr= model.sell['B',1] <= model.seats['B'])
model.demandB_2 = pyo.Constraint(expr= model.sell['B',2] <= 25)
model.limitB_2 = pyo.Constraint(expr= model.sell['B',2] <= model.seats['B'])
model.demandB_3 = pyo.Constraint(expr= model.sell['B',3] <= 10)
model.limitB_3 = pyo.Constraint(expr= model.sell['B',3] <= model.seats['B'])
model.demandE_1 = pyo.Constraint(expr= model.sell['E',1] <= 200)
model.limitE_1 = pyo.Constraint(expr= model.sell['E',1] <= model.seats['E'])
model.demandE_2 = pyo.Constraint(expr= model.sell['E',2] <= 175)
model.limitE_2 = pyo.Constraint(expr= model.sell['E',2] <= model.seats['E'])
model.demandE_3 = pyo.Constraint(expr= model.sell['E',3] <= 150)
model.limitE_3 = pyo.Constraint(expr= model.sell['E',3] <= model.seats['E'])

def second_stage_profit(model):
    total_1 = model.sell['F',1] * model.pricefactor_F + model.sell['B',1] * model.pricefactor_B + model.sell['E',1]
    total_2 = model.sell['F',2] * model.pricefactor_F + model.sell['B',2] * model.pricefactor_B + model.sell['E',2]
    total_3 = model.sell['F',3] * model.pricefactor_F + model.sell['B',3] * model.pricefactor_B + model.sell['E',3]
    return (total_1 + total_2 + total_3)/3.0

model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

def total_profit(model):
    return model.second_stage_profit

model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

result = cbc_solver.solve(model)
display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
display(Markdown(f"**Solution:**"))
display(Markdown(f"(seat allocation) $x_F = {model.seats['F'].value:.0f}$, $e_B = {model.seats['B'].value:.0f}$, $e_E = {model.seats['E'].value:.0f}$"))
display(Markdown(f"(equivalent seat allocation) $e_F = {model.equivalentseatsF.expr():.0f}$, $e_B = {model.equivalentseatsB.expr():.0f}$, $e_E = {model.equivalentseatsE.expr():.0f}$"))
display(Markdown(f"(recourse sell action scenario 1) $s_F = {model.sell['F',1].value:.0f}$, $s_B = {model.sell['B',1].value:.0f}$, $s_E = {model.sell['E',1].value:.0f}$"))
display(Markdown(f"(recourse sell action scenario 2) $s_F = {model.sell['F',2].value:.0f}$, $s_B = {model.sell['B',2].value:.0f}$, $s_E = {model.sell['E',2].value:.0f}$"))
display(Markdown(f"(recourse sell action scenario 3) $s_F = {model.sell['F',3].value:.0f}$, $s_B = {model.sell['B',3].value:.0f}$, $s_E = {model.sell['E',3].value:.0f}$"))
display(Markdown(f"**Optimal objective value:** ${model.total_expected_profit():.0f}$ (in units of economy ticket price)"))

**Solver status:** *ok, optimal*

**Solution:**

(seat allocation) $x_F = 10$, $e_B = 20$, $e_E = 150$

(equivalent seat allocation) $e_F = 20$, $e_B = 30$, $e_E = 150$

(recourse sell action scenario 1) $s_F = 10$, $s_B = 20$, $s_E = 150$

(recourse sell action scenario 2) $s_F = 10$, $s_B = 20$, $s_E = 150$

(recourse sell action scenario 3) $s_F = 5$, $s_B = 10$, $s_E = 150$

**Optimal objective value:** $208$ (in units of economy ticket price)

Assume now that the airline wishes a special guarantee for its clients enrolled in its loyalty program. In particular, it wants $98\%$ probability to cover the demand of first-class seats and $95\%$ probability to cover the demand of business class seats (by clients of the loyalty program). First-class passengers are covered if they get a first-class seat. Business class passengers are covered if they get either a business or a first-class seat (upgrade). Assume weekday demands of loyalty-program passengers are normally distributed, say $\xi_F \sim \mathcal N(16,16)$ and $\xi_B \sim \mathcal N(30,48)$ for first-class and business, respectively. Also assume that the demands for first-class and business class seats are independent.
Let $x_1$ be the number of first-class seats and $x_2$ the number of business seats. The probabilistic constraints are simply

$$
	\mathbb P(x_1 \geq \xi_F ) \geq 0.98, \qquad \text{ and } \qquad \mathbb P(x_1 +x_2 \geq \xi_F + \xi_B ) \geq 0.95.
$$

In Exercise 3 of the tutorial you rewrote these equivalently as linear constraints, specifically 

$$
	(x_1 - 16)/\sqrt{16} \geq 2.054 \qquad \text{ and } \qquad (x_1 +x_2 - 46)/ \sqrt{64} \geq 1.645.
$$

(b) Add to your implementation of the extensive form the two equivalent deterministic constraints corresponding to the two chance constraints and find the new optimal solution meeting these additional constraints. How is it different from the previous one?

In [ ]:
model = pyo.ConcreteModel()

model.classes = pyo.Set(initialize=['F', 'B', 'E'])
model.totalseats = 200
model.pricefactor_F = 3.0
model.pricefactor_B = 2.0
model.seatfactor_F = 2.0 
model.seatfactor_B = 1.5

# first stage variables
model.seats = pyo.Var(model.classes, within=pyo.NonNegativeIntegers) 

# first stage constraint
model.equivalentseatsF = pyo.Expression(expr=model.seats['F']*model.seatfactor_F)
model.equivalentseatsB = pyo.Expression(expr=model.seats['B']*model.seatfactor_B)
model.equivalentseatsE = pyo.Expression(expr=model.seats['E'])
model.planeseats = pyo.Constraint(expr=model.equivalentseatsF + model.equivalentseatsB + model.equivalentseatsE <= model.totalseats)
model.loyaltyF = pyo.Constraint(expr = model.seats['F'] >= 24.22) #loyalty constraint 1
model.loyaltyFB = pyo.Constraint(expr = model.seats['F']+model.seats['B'] >= 59.16)  #loyalty constraint 2

model.scenarios = pyo.Set(initialize=[1,2,3])

# second stage variables (labelled as 1,2,3 depending on the scenario)
model.sell = pyo.Var(model.classes, model.scenarios, within=pyo.NonNegativeIntegers)

# second stage constraints
model.demandF_1 = pyo.Constraint(expr= model.sell['F',1] <= 20)
model.limitF_1 = pyo.Constraint(expr= model.sell['F',1] <= model.seats['F'])
model.demandF_2 = pyo.Constraint(expr= model.sell['F',2] <= 10)
model.limitF_2 = pyo.Constraint(expr= model.sell['F',2] <= model.seats['F'])
model.demandF_3 = pyo.Constraint(expr= model.sell['F',3] <= 5)
model.limitF_3 = pyo.Constraint(expr= model.sell['F',3] <= model.seats['F'])
model.demandB_1 = pyo.Constraint(expr= model.sell['B',1] <= 50)
model.limitB_1 = pyo.Constraint(expr= model.sell['B',1] <= model.seats['B'])
model.demandB_2 = pyo.Constraint(expr= model.sell['B',2] <= 25)
model.limitB_2 = pyo.Constraint(expr= model.sell['B',2] <= model.seats['B'])
model.demandB_3 = pyo.Constraint(expr= model.sell['B',3] <= 10)
model.limitB_3 = pyo.Constraint(expr= model.sell['B',3] <= model.seats['B'])
model.demandE_1 = pyo.Constraint(expr= model.sell['E',1] <= 200)
model.limitE_1 = pyo.Constraint(expr= model.sell['E',1] <= model.seats['E'])
model.demandE_2 = pyo.Constraint(expr= model.sell['E',2] <= 175)
model.limitE_2 = pyo.Constraint(expr= model.sell['E',2] <= model.seats['E'])
model.demandE_3 = pyo.Constraint(expr= model.sell['E',3] <= 150)
model.limitE_3 = pyo.Constraint(expr= model.sell['E',3] <= model.seats['E'])

def second_stage_profit(model):
    total_1 = model.sell['F',1] * model.pricefactor_F + model.sell['B',1] * model.pricefactor_B + model.sell['E',1]
    total_2 = model.sell['F',2] * model.pricefactor_F + model.sell['B',2] * model.pricefactor_B + model.sell['E',2]
    total_3 = model.sell['F',3] * model.pricefactor_F + model.sell['B',3] * model.pricefactor_B + model.sell['E',3]
    return (total_1 + total_2 + total_3)/3.0

model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

def total_profit(model):
    return model.second_stage_profit

model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

result = cbc_solver.solve(model)
display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
display(Markdown(f"**Solution:**"))
display(Markdown(f"(seat allocation) $x_F = {model.seats['F'].value:.0f}$, $e_B = {model.seats['B'].value:.0f}$, $e_E = {model.seats['E'].value:.0f}$"))
display(Markdown(f"(equivalent seat allocation) $e_F = {model.equivalentseatsF.expr():.0f}$, $e_B = {model.equivalentseatsB.expr():.0f}$, $e_E = {model.equivalentseatsE.expr():.0f}$"))
display(Markdown(f"(recourse sell action scenario 1) $s_F = {model.sell['F',1].value:.0f}$, $s_B = {model.sell['B',1].value:.0f}$, $s_E = {model.sell['E',1].value:.0f}$"))
display(Markdown(f"(recourse sell action scenario 2) $s_F = {model.sell['F',2].value:.0f}$, $s_B = {model.sell['B',2].value:.0f}$, $s_E = {model.sell['E',2].value:.0f}$"))
display(Markdown(f"(recourse sell action scenario 3) $s_F = {model.sell['F',3].value:.0f}$, $s_B = {model.sell['B',3].value:.0f}$, $s_E = {model.sell['E',3].value:.0f}$"))
display(Markdown(f"**Optimal objective value:** ${model.total_expected_profit():.0f}$ (in units of economy ticket price)"))

**Solver status:** *ok, optimal*

**Solution:**

(seat allocation) $x_F = 25$, $e_B = 35$, $e_E = 97$

(equivalent seat allocation) $e_F = 50$, $e_B = 52$, $e_E = 97$

(recourse sell action scenario 1) $s_F = 20$, $s_B = 35$, $s_E = 97$

(recourse sell action scenario 2) $s_F = 10$, $s_B = 25$, $s_E = 97$

(recourse sell action scenario 3) $s_F = 5$, $s_B = 10$, $s_E = 97$

**Optimal objective value:** $179$ (in units of economy ticket price)

Assume now that the ticket demand for the three categories is captured by a $3$-dimensional multivariate normal with mean $\mu=(16,30,180)$ and covariance matrix 
$$
\Sigma= \left(
\begin{array}{ccc}
 3.5 & 3.7 & 2.5 \\
 3.7 & 6.5 & 7.5 \\
 2.5 & 7.5 & 25.2 \\
\end{array}
\right).
$$

(c) Solve approximately the airline seat allocation problem (with the loyalty constraints) using the Sample Average Approximation method. More specifically, sample $N=1000$ points from the multivariate normal distribution and solve the extensive form for the stochastic LP resulting from those $N=1000$ scenarios.

In [ ]:
def AirlineSAA(N, sample):

    model = pyo.ConcreteModel()

    def indices_rule(model):
        return range(N)

    model.scenarios = pyo.Set(initialize=indices_rule)
    model.demandF = pyo.Param(model.scenarios, initialize=dict(enumerate(sample[:,0])))
    model.demandB = pyo.Param(model.scenarios, initialize=dict(enumerate(sample[:,1])))
    model.demandE = pyo.Param(model.scenarios, initialize=dict(enumerate(sample[:,2])))

    model.classes = pyo.Set(initialize=['F', 'B', 'E'])
    model.totalseats = 200
    model.pricefactor_F = 3.0
    model.pricefactor_B = 2.0
    model.seatfactor_F = 2.0 
    model.seatfactor_B = 1.5

    # first stage variables
    model.seats = pyo.Var(model.classes, within=pyo.NonNegativeIntegers) 

    # first stage constraint
    model.equivalentseatsF = pyo.Expression(expr=model.seats['F']*model.seatfactor_F)
    model.equivalentseatsB = pyo.Expression(expr=model.seats['B']*model.seatfactor_B)
    model.equivalentseatsE = pyo.Expression(expr=model.seats['E'])
    model.planeseats = pyo.Constraint(expr=model.equivalentseatsF + model.equivalentseatsB + model.equivalentseatsE <= model.totalseats)
    model.loyaltyF = pyo.Constraint(expr = model.seats['F'] >= 24.22)
    model.loyaltyFB = pyo.Constraint(expr = model.seats['F']+model.seats['B'] >= 59.16)

    # second stage variables
    model.sell = pyo.Var(model.classes, model.scenarios, within=pyo.NonNegativeIntegers)

    # second stage constraints
    model.demandFlim = pyo.ConstraintList()
    model.limitF = pyo.ConstraintList()
    model.demandBlim = pyo.ConstraintList()
    model.limitB = pyo.ConstraintList()
    model.demandElim = pyo.ConstraintList()
    model.limitE = pyo.ConstraintList()
    for i in model.scenarios:
        model.demandFlim.add(expr= model.sell['F',i] <= model.demandF[i])
        model.limitF.add(expr= model.sell['F',i] <= model.seats['F'])
        model.demandBlim.add(expr= model.sell['B',i] <= model.demandB[i])
        model.limitB.add(expr= model.sell['B',i] <= model.seats['B'])
        model.demandElim.add(expr= model.sell['E',i] <= model.demandE[i])
        model.limitE.add(expr= model.sell['E',i] <= model.seats['E'])

    def second_stage_profit(model):
        return sum([model.sell['F',i] * model.pricefactor_F + model.sell['B',i] * model.pricefactor_B + model.sell['E',i] for i in model.scenarios])/float(N)

    model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

    def total_profit(model):
        return model.second_stage_profit

    model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

    result = cbc_solver.solve(model)
    display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
    display(Markdown(f"**Solution:**"))
    display(Markdown(f"(seat allocation) $x_F = {model.seats['F'].value:.0f}$, $e_B = {model.seats['B'].value:.0f}$, $e_E = {model.seats['E'].value:.0f}$"))
    display(Markdown(f"(equivalent seat allocation) $e_F = {model.equivalentseatsF.expr():.0f}$, $e_B = {model.equivalentseatsB.expr():.0f}$, $e_E = {model.equivalentseatsE.expr():.0f}$"))
    display(Markdown(f"**Optimal objective value:** ${model.total_expected_profit():.0f}$ (in units of economy ticket price)"))

N = 1000
np.random.seed(1)
samples = np.random.multivariate_normal([16, 30, 180],[[3.5, 3.7, 2.5],[3.7, 6.5, 7.5],[2.5, 7.5, 25.2]], N)
AirlineSAA(N, samples)

**Solver status:** *ok, optimal*

**Solution:**

(seat allocation) $x_F = 25$, $e_B = 35$, $e_E = 97$

(equivalent seat allocation) $e_F = 50$, $e_B = 52$, $e_E = 97$

**Optimal objective value:** $202$ (in units of economy ticket price)